## 🎯 Objectif : segmentation client
Identifier des profils types de clients selon leur comportement d’achat, leur profil sociodémographique ou leurs préférences produit.

### 1. Import Package and data

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, 36, Finished, Available, Finished)

In [35]:
# Load data into pandas DataFrame from "/lakehouse/default/Files/retail_sales_dataset.csv"
df = pd.read_csv("/lakehouse/default/Files/retail_sales_dataset.csv")
display(df)

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, 37, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2d54d88f-310b-42ff-b680-e6dcfe14f6ae)

### 2. Preprocessing


In [36]:
# standardize column names
df.columns = df.columns.str.lower().str.replace(' ','_')
df.head()

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, 38, Finished, Available, Finished)

,transactionid,date,customerid,gender,age,productcategory,quantity,priceperunit,totalamount
0,1,24/11/2023,CUST001,Male,34,Beauty,3,50,150
1,2,27/02/2023,CUST002,Female,26,Clothing,2,500,1000
2,3,13/01/2023,CUST003,Male,50,Electronics,1,30,30
3,4,21/05/2023,CUST004,Male,37,Clothing,1,500,500
4,5,06/05/2023,CUST005,Male,30,Beauty,2,50,100


In [37]:
df['date'] = pd.to_datetime(df['date'])

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, 39, Finished, Available, Finished)

In [38]:
print(df['gender'].value_counts(), '\n')
print(df['productcategory'].value_counts())

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, 40, Finished, Available, Finished)

gender
Female    510
Male      490
Name: count, dtype: int64 

productcategory
Clothing       351
Electronics    342
Beauty         307
Name: count, dtype: int64


In [39]:
print(df.customerid.value_counts().max())

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, 41, Finished, Available, Finished)

1


There are no repeat purchases. Each customer has only one transaction. To make our RFM customer segmentation practice fun and meaningful, let's introduce multiple purchases for each customer. We re-create the customer_id column and fill a random number between 1 to 1000.

In [40]:
np.random.seed(101)

df['customer_id_new'] = np.random.randint(0, 500, size=1000)
df.head()

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, 42, Finished, Available, Finished)

,transactionid,date,customerid,gender,age,productcategory,quantity,priceperunit,totalamount,customer_id_new
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,351
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,11
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,337
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,326
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,63


In [41]:
df.customer_id_new.value_counts()

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, 43, Finished, Available, Finished)

customer_id_new
383    7
343    7
133    6
424    6
215    6
      ..
43     1
439    1
413    1
412    1
202    1
Name: count, Length: 429, dtype: int64

In [42]:
df['customer_id_new'] = df['customer_id_new'].apply(lambda x: f'CUST{str(x).zfill(3)}')

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, 44, Finished, Available, Finished)

In [43]:
df['customerid'] = df['customer_id_new']
df.head()

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, 45, Finished, Available, Finished)

,transactionid,date,customerid,gender,age,productcategory,quantity,priceperunit,totalamount,customer_id_new
0,1,2023-11-24,CUST351,Male,34,Beauty,3,50,150,CUST351
1,2,2023-02-27,CUST011,Female,26,Clothing,2,500,1000,CUST011
2,3,2023-01-13,CUST337,Male,50,Electronics,1,30,30,CUST337
3,4,2023-05-21,CUST326,Male,37,Clothing,1,500,500,CUST326
4,5,2023-05-06,CUST063,Male,30,Beauty,2,50,100,CUST063


In [44]:
df = df.drop(columns=['customer_id_new'])
df.customerid.value_counts().head()

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, 46, Finished, Available, Finished)

customerid
CUST383    7
CUST343    7
CUST133    6
CUST424    6
CUST215    6
Name: count, dtype: int64

In [46]:
# Étape 1 : regrouper les infos client
customer_infos_clean = df.groupby('customerid').agg({
    'gender': 'first',              
    'age': 'first',                 
    'totalamount': 'sum',          
    'date': 'max',                 
    'transactionid': 'nunique'     
}).reset_index()

# Étape 2 : renommer proprement les colonnes
customer_infos_clean.rename(columns={
    'customerid': 'customer_id',
    'gender': 'gender',
    'age': 'age',
    'totalamount': 'total_spent',
    'date': 'last_order_date',
    'transactionid': 'nb_transactions'
}, inplace=True)

# ✅ Étape 3 : créer la colonne age_group
def assign_age_group(age):
    if age < 25:
        return "Moins de 25"
    elif age < 40:
        return "25-39"
    elif age < 60:
        return "40-59"
    else:
        return "60+"

customer_infos_clean['age_group'] = customer_infos_clean['age'].apply(assign_age_group)

# Étape 4 : vérification des doublons
duplicates = customer_infos_clean.duplicated(subset='customer_id')
print(f"Nombre de doublons trouvés : {duplicates.sum()}")

# Étape 5 : export dans Fabric
spark_df_infos = spark.createDataFrame(customer_infos_clean)
spark_df_infos.write.mode("overwrite").format("delta").option("mergeSchema", "true").saveAsTable("dataset2.customer_infos")


StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, 48, Finished, Available, Finished)

Nombre de doublons trouvés : 0


### 4. RFM Customer Segmentation

RFM = Recency, Frequency, Monetary

Critère	Signification	Exemples <br>
- R (Recency) Depuis combien de temps le client a acheté	5 jours, 60 jours…<br>
- F (Frequency)	Combien de fois il a acheté	1, 3, 10 achats sur la période<br>
- M (Monetary)	Combien il a dépensé au total	20 €, 300 €, 1000 €…<br>
<br>
🎯 Pourquoi utiliser RFM ?
- Identifier les clients fidèles et rentables
- Détecter les clients à réactiver
- Cibler les meilleurs segments dans ton marketing
- Générer un clustering non supervisé simple et très business-friendly



In [ ]:
# Tu cherches la date la plus récente dans les transactions. Elle sert à calculer la récence (depuis combien de jours un client a fait son dernier achat).

max_date = df['date'].max()
max_date

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, -1, Cancelled, , Cancelled)

In [ ]:
# create customer-level dataset
# Étape 2 – Créer une table client agrégée
# Tu regroupes les transactions par client (customerid) pour calculer :
# recency = nombre de jours depuis la dernière transaction
# frequency = nombre de transactions
# monetary = total dépensé (somme de totalamount)


customer_data = df.groupby('customerid').agg({
    'date': lambda x: (max_date - x.max()).days,  # recency
    'transactionid': 'count',  # frequency
    'totalamount': 'sum'  # monetary
})

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, -1, Cancelled, , Cancelled)

In [ ]:
#Renommer les colonnes

customer_data.rename(columns={
    'date': 'recency',
    'transactionid': 'frequency',
    'totalamount': 'monetary'
}, inplace=True)

customer_data.head()


StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, -1, Cancelled, , Cancelled)

In [ ]:
# assign scores to each RFM component
# Calcul des quantiles
# Tu calcules les seuils (quartiles) de chaque colonne R, F, M : 25%, 50%, 75% → utiles pour donner un score de 1 à 5 à chaque client selon où il se situe par rapport aux autres.

quantiles = customer_data.quantile(q=[.25, .5, .75])
quantiles = quantiles.to_dict()

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, -1, Cancelled, , Cancelled)

In [ ]:
quantiles

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, -1, Cancelled, , Cancelled)

Ce qui est bien :
- x est le nombre de jours depuis le dernier achat (donc plus c’est petit, mieux c’est)

Tu attribues un score :

- 5 = client très récent
- 1 = client très ancien



In [ ]:
def RScore(x):
    if x <= quantiles['recency'][0.25]:
        return 5
    elif x <= quantiles['recency'][0.5]:
        return 4
    elif x <= quantiles['recency'][0.75]:
        return 3
    else:
        return 2 if x <= customer_data['recency'].max() else 1

def FScore(x):
    if x <= quantiles['frequency'][0.25]:
        return 1
    elif x <= quantiles['frequency'][0.5]:
        return 2
    elif x <= quantiles['frequency'][0.75]:
        return 3
    else:
        return 4 if x <= customer_data['frequency'].max() else 5

def MScore(x):
    if x <= quantiles['monetary'][0.25]:
        return 1
    elif x <= quantiles['monetary'][0.5]:
        return 2
    elif x <= quantiles['monetary'][0.75]:
        return 3
    else:
        return 4 if x <= customer_data['monetary'].max() else 5

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, -1, Cancelled, , Cancelled)

In [ ]:
customer_data['R_Score'] = customer_data['recency'].apply(RScore)
customer_data['F_Score'] = customer_data['frequency'].apply(FScore)
customer_data['M_Score'] = customer_data['monetary'].apply(MScore)

customer_data['RFM_Score'] = (
    customer_data['R_Score'].astype(str) +
    customer_data['F_Score'].astype(str) +
    customer_data['M_Score'].astype(str)
)


StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, -1, Cancelled, , Cancelled)

In [ ]:
# Définition du segment client
def segment(rfm):
    if rfm == "555":
        return "Champion"
    elif rfm.startswith("5"):
        return "Loyal"
    elif rfm.endswith("5"):
        return "Big Spender"
    elif rfm == "111":
        return "Lost"
    else:
        return "Other"

customer_data['Segment'] = customer_data['RFM_Score'].apply(segment)

# Affichage final
customer_data.head()

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, -1, Cancelled, , Cancelled)

In [ ]:
plt.figure(figsize=(10,4))
rfm_level_counts = customer_data['RFM_Score'].value_counts().sort_index()
sns.barplot(x=rfm_level_counts.index, y=rfm_level_counts.values)
plt.title('Distribution of RFM Scores')
plt.xlabel('RFM Score')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()


StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, -1, Cancelled, , Cancelled)

### K-means clustering

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(customer_data[['R_Score', 'F_Score', 'M_Score']])

# Calcul de l'inertie pour différents nombres de clusters
inertia = []
K = range(1, 11)

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(rfm_scaled)
    inertia.append(kmeans.inertia_)

# Affichage de la courbe du coude (elbow method)
plt.figure(figsize=(6, 4))
plt.plot(K, inertia, 'bo-')
plt.xlabel('Nombre de clusters (k)')
plt.ylabel('Inertie')
plt.title('Méthode du coude pour choisir k optimal')
plt.grid(True)
plt.tight_layout()
plt.show()


StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, -1, Cancelled, , Cancelled)

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42)
customer_data['Cluster'] = kmeans.fit_predict(rfm_scaled)

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, -1, Cancelled, , Cancelled)

In [ ]:
cluster_summary = customer_data.groupby('Cluster').agg({
    'R_Score': 'mean',
    'F_Score': 'mean',
    'M_Score': 'mean'
}).reset_index()


StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, -1, Cancelled, , Cancelled)

In [ ]:
cluster_summary


StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, -1, Cancelled, , Cancelled)

In [ ]:
print(customer_data.columns.tolist())


StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, -1, Cancelled, , Cancelled)

In [ ]:
# Assigning the segment names
segment_names = {
    0: 'At Risk',
    1: 'Recent Customers',
    2: 'Big Spenders',
    3: 'Frequent Low Spenders'
}

# Assign the segment names to the Cluster
customer_data['Segment'] = customer_data['Cluster'].map(segment_names)

# Display the customer data with segments
# df_rfm_final = customer_data[['R_Score', 'F_Score', 'M_Score', 'RFM_Score', 'Cluster', 'Segment']]
df_rfm_final = customer_data.reset_index()[[
    'customerid',
    'recency',
    'frequency',
    'monetary',
    'R_Score', 'F_Score', 'M_Score',
    'RFM_Score',
    'Cluster',
    'Segment'
]]
df_rfm_final.rename(columns={'customerid': 'customer_id'}, inplace=True)

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, -1, Cancelled, , Cancelled)

In [ ]:
spark_df_rfm = spark.createDataFrame(df_rfm_final)
spark_df_rfm.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .format("delta") \
    .saveAsTable("dataset2.customer_rfm_segmentation")

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, -1, Cancelled, , Cancelled)

In [ ]:
# size of each segment
segment_size = customer_data['Segment'].value_counts().reset_index()
segment_size.columns = ['Segment','Count']
segment_size

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, -1, Cancelled, , Cancelled)

In [ ]:
plt.figure(figsize=(7,3))
sns.countplot(data=customer_data, x='Segment', order=segment_size['Segment'])
plt.title('Customer Segments Distribution')
plt.xlabel('Segment')
plt.ylabel('Number of Customers')
plt.xticks(rotation=45)
plt.show()


StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, -1, Cancelled, , Cancelled)

In [47]:
df_customer_full = pd.merge(
    customer_infos_clean,
    df_rfm_final,
    on='customer_id',
    how='inner'
)

print(f"Nombre de lignes après merge : {df_customer_full.shape[0]}")

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, 49, Finished, Available, Finished)

Nombre de lignes après merge : 429


In [49]:
spark_df_full = spark.createDataFrame(df_customer_full)
spark_df_full.write.mode("overwrite").format("delta").option("mergeSchema", "true").saveAsTable("dataset2.customer_segmentation")

StatementMeta(, d27f73c5-3d57-4b8c-9ce0-285bf2c9a386, 51, Finished, Available, Finished)